In [1]:
import os
import geopandas as gp
import pandas as pd
import numpy as np
import uuid

In [2]:
def changecat(utility):
    a=pd.DataFrame(pd.read_excel(r"NTDB_V2_DOMAIN_07012020.xlsx", sheet_name='LU-LC-CADASTRAL'))
    a.columns = a.iloc[0]
    a[["CATEGORY","FEATURE"]] = a[["CATEGORY","FEATURE"]].fillna(method='pad')
    b = a.loc[:,['SUBTYPE']]
    a["SUBTYPE"]=a["SUBTYPE"].fillna(" ")
    if a["SUBTYPE"].dropna().str.contains(utility).sum()>0:
        fea=a[a["SUBTYPE"].dropna().str.contains(utility)]["FEATURE"].unique()
        cat=a[a["SUBTYPE"].dropna().str.contains(utility)]["CATEGORY"].unique()
        return fea,cat[0]
    else:
        return None, None

In [3]:
def getcatname(feat):
    a=pd.DataFrame(pd.read_excel(r"NTDB_V2_DOMAIN_07012020.xlsx", sheet_name='LU-LC-CADASTRAL'))
    a.columns = a.iloc[0]
    a[["CATEGORY","FEATURE"]] = a[["CATEGORY","FEATURE"]].fillna(method='pad')
    b = a.loc[:,['SUBTYPE']]
    a["SUBTYPE"]=a["SUBTYPE"].fillna(" ")
    cat=a[a["FEATURE"].dropna().str.contains(feat)]["CATEGORY"].unique()
    if(len(cat)>0):
        return cat[0]
    else:
        return

In [ ]:
excel=pd.DataFrame(pd.read_excel(r"NTDB_V2_ATTRIBUTES_07012020.xlsx", sheet_name='LU-LC-CADASTRAL'))
excel =excel.fillna(method='pad').set_index("CATEGORY")
feature=excel['FEATURE'].dropna().unique()
excel=excel.T
he=list(excel.columns.values)
header=[he,excel.loc['FEATURE'],excel.loc['ATTRIBUTE NAME']]
excel.columns=header
excel=excel.iloc[0:0]
fe=input("ENTER THE NAME OF THE FEATURE: ")
cat=getcatname(fe)
if(cat is not None):
    a=excel[cat][fe]
    filepath="Point of Interest data of two districts Raygad and Pune of Maharashtra ( Sahyog data)"
    ind=0
    for pa,k,j in os.walk(filepath):
        for mi in j:
            if(mi.endswith(".dbf")):
                fea,cat=changecat(mi[:-4])
                if(fea is not None):
                    if(fea[0]==fe):
                        d=pd.DataFrame(gp.read_file(os.path.join(pa,mi)))
                        colh=d.columns.values
                        colm=a.columns.values
                        for i in colh:
                            if(i=='POI NAME' or i=='POI_NAME' or i=='NAME OF ' or i=='Name'):
                                d.rename(columns={i:'NAME'},inplace=True)
                            elif(i=='c_name'):
                                d.rename(columns={i:'CATEGORY'},inplace=True)
                            elif(i=='survey_d'):
                                d.rename(columns={i:'SUR_DATE'},inplace=True)
                            elif(i=='ADDITION' or i=='ADL_INFO' or i=='BANK'):
                                d.rename(columns={i:'ADL_INFO'},inplace=True)
                            elif(i=='utility_'):
                                d.rename(columns={i:'CLSFD_AS'},inplace=True)
                            elif(i=='CONTACT_'):
                                d.rename(columns={i:'CONTACTNO'},inplace=True)
                            elif(i=='Type'):
                                d.rename(columns={i:'TYPE'},inplace=True)
                            elif(i=='SURVEY N' or i=='SURVEY_N'):
                                d.rename(columns={i:'SUR_NO'},inplace=True)
                            elif('ADDRESS' in colm):
                                if(i=='STREET N' or i=='LOCALITY' or i=='BUILDING'):
                                    d.rename(columns={i:'ADDRESS'},inplace=True)
                            elif('ADDRESS' not in colh):
                                if(i=='STREET N' or i=='LOCALITY' or i=='BUILDING'):
                                    d.rename(columns={i:'ADL_INFO'},inplace=True)
                        colh=d.columns.values.tolist()
                        d.drop_duplicates()
                        for m in colh:
                            m=str(m)
                            ind=len(d[m])
                            if colh.count(m)>1:
                                temp=colh.count(m)
                                def sjoin(x): return ';'.join(x[x.notnull()].astype(str))
                                d['a']=d[m].groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))
                                d.drop(m,axis=1,inplace=True)
                                for it in range(temp-1):
                                    colh.remove(m)
                                    d.rename(columns={"a":m},inplace=True)
                        colh=d.columns.values.tolist()
                        for m in colh:
                            for n in range(ind):
                                if(pd.isna(d[m][n])==False ):
                                    if(len(str(d[m][n]))<3):
                                        d[m][n]=np.nan
                                    if(m=='CONTACT_' and len(str(d[m][n]))<10):
                                        d[m][n]=np.nan
                                    if(m=='TYPE' and "NAME" in colh):
                                        if(d[m][n]==d['NAME'][n]):
                                            d[m][n]=np.nan
                                        elif(d[m][n]==d['CLSFD_AS'][n]):
                                            d[m][n]=np.nan
                                    if(m=='CLSFD_AS' and "NAME" in colh):
                                        if(d[m][n]==d['NAME'][n]):
                                            if(n-1<0):
                                                d[m][n]=np.nan
                                            else:
                                                d[m][n]=d[m][n-1]
                        colm=a.columns.values.tolist()
                        for col in colh:
                            if col not in colm:
                                d=d.drop([col],axis=1)
                        a=a.append(d,ignore_index=True)
    a['UNIQ_ID'] = a.apply(lambda _: uuid.uuid4(), axis=1)
    a.to_csv(fe+".csv")
    print(fe+".csv FILE HAS BEEN GENERATED")
    del a
else:
    print("THIS FEATURE DOES NOT EXIST")